In [ ]:
!pip install rich transformers torch accelerate bitsandbytes peft

Task: Text Generation

Each task has its own default model in the pipeline.

- Causal Language Modeling

- Masked Language Modeling

Another type of variation is

- Text Generation

- Text-to-Text Generation models


Variety of LMs in HuggingFace

https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard


Some of the Text generation Tasks

Code Generation: Trained to generate code

https://huggingface.co/spaces/bigcode/bigcode-playground

Instruction Model: Those that are trained on instruction

Stories generation: A prompt starts the Stories Generation


Quantization Using BitsAndBytes is touched

- Grokking how the models are shrunk and loaded.



In [ ]:
# there is default chat for the model, so it must give output
roberta_tokenizer.default_chat_template


No chat template is defined for this tokenizer - using a default chat template that implements the ChatML format (without BOS/EOS tokens!). If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



"{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"

In [ ]:
import random

# get the questions
questions = [q for q in dataset['instruction']]

random.shuffle(questions)  # no need to reassign

prompt_template = """Below is a riddle. Come up with 10 more.
Output just the riddles. No numbering and don't output anything else"""

simple_request = "Generate 10 riddles that you know"

prompt_riddles = prompt_template + "\n\n".join(questions[0:10])

# need to use join, list and string dont concatenate

messages_to_model = [{"role":"user", "content":prompt_riddles}]

simple_message = [{"role":"user", "content":simple_request}]

In [ ]:
# roberta_input = roberta_tokenizer.apply_chat_template(messages_to_roberta, return_tensors='pt').to('cuda')
roberta_input = roberta_tokenizer.apply_chat_template(simple_message,
                                                      return_tensors='pt').to('cuda')
roberta_input

tensor([[41552, 15483,   757,  1215, 13124, 15483, 15698, 12105, 50118, 40025,
           877,   158,   910, 40741,    14,    47,   216, 41552, 15483,   757,
          1215,  1397, 15483, 15698, 50118]], device='cuda:0')

In [ ]:
roberta_tokenizer.decode(roberta_input[0])

'<|im_start|>user\nGenerate 10 riddles that you know<|im_end|>\n'

In [ ]:
roberta.generation_config

GenerationConfig {
  "bos_token_id": 0,
  "eos_token_id": 2,
  "pad_token_id": 1
}

In [ ]:
roberta_output = roberta.generate(
    roberta_input,
    max_new_tokens=50,
    do_sample=True,
    pad_token_id=roberta_tokenizer.eos_token_id
)

In [ ]:
output = roberta_output[0][len(roberta_input[0]):]
output

tensor([2], device='cuda:0')

In [ ]:
output_text = roberta_tokenizer.decode(output, skip_special_tokens=True)
output_text # There is no output at all. checking why, with a simpler prompt

''

In [ ]:
del roberta  # removes the memory handles

In [ ]:
torch.cuda.empty_cache()  # garbage collects the memory

Diving into BitsAndBytes Configuration.

BitsAndBytesConfig(
     load_in_8bit=False,
  
      load_in_4bit=False,

       llm_int8_threshold=6.0,

        llm_int8_skip_modules=None,
    
    llm_int8_enable_fp32_cpu_offload=False,
    
    llm_int8_has_fp16_weight=False
    ,
    bnb_4bit_compute_dtype=Non
    e,
    bnb_4bit_quant_type='fp
    4',
    bnb_4bit_use_double_quant=Fa
    lse,
    **kwargs,
)

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

#### Working on llama2-7b

In [ ]:
# llama_path = "/home/kamal/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-chat-hf/snapshots/c1b0db933684edbfe29a06fa47eb19cc48025e93/"

gemma_path = "google/gemma-2b-it"
gemma = AutoModelForCausalLM.from_pretrained(
    # pretrained_model_name_or_path='/home/aicoder/.cache/huggingface/hub/models--google--gemma-2b-it/snapshots/718cb189da9c5b2e55abe86f2eeffee9b4ae0dad/
    gemma_path,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    local_files_only=True
) # takes 11GB of VRAM

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
gemma_tokenizer = AutoTokenizer.from_pretrained(gemma_path)
# print(llama_tokenizer.default_chat_template) # rich's print fails due to tag
gemma_tokenizer.default_chat_template

"{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"

In [ ]:
gemma_tokenizer.special_tokens_map

{'bos_token': '<bos>',
 'eos_token': '<eos>',
 'unk_token': '<unk>',
 'pad_token': '<pad>',
 'additional_special_tokens': ['<start_of_turn>', '<end_of_turn>']}

In [ ]:
gemma_input = llama_tokenizer.apply_chat_template(simple_message,
                                                  return_tensors='pt').to('cuda')
gemma_input

tensor([[     2,    106,   1645,    108,  38557, 235248, 235274, 235276, 193130,
            674,    692,   1230,    107,    108]], device='cuda:0')

In [ ]:
gemma_output = gemma.generate(
    gemma_input,
    max_new_tokens=100,
    do_sample=True,
    pad_token_id = llama_tokenizer.eos_token_id
)

In [ ]:
# llama_output = llama_output[len(llama_input[0]):]
# llama_output
output = gemma_tokenizer.decode(gemma_output[0],skip_special_tokens=True)
output

'user\nGenerate 10 riddles that you know\n1. I have keys, but no doors.\nI have space, but no room.\nI have pages, but no words.\n\nWhat am I?\n\n\n2. I have a bed, but I never sleep.\nI have a mouth, but I never speak.\nI have ears, but I never hear.\n\nWhat am I?\n\n\n3. I have a tongue, but I never eat.\nI have a head, but I never wear.\nI have a'

In [ ]:
# working on getting the riddle based on the 10 riddles input
gemma_10_input = gemma_tokenizer.apply_chat_template(messages_to_model,
                                                     return_tensors='pt').to('cuda')
gemma_10_input

tensor([[     2,    106,   1645,    108,  33501,    603,    476, 133326, 235265,
          12542,    908,    675, 235248, 235274, 235276,    978, 235265,    108,
           6140,   1317,    573, 193130, 235265,   1307,  96085,    578,   1453,
         235303, 235251,   5033,   4341,   1354, 235285,  25469,    578,  10084,
            578,  47331,   2003,    575,    861,   3142, 235265,    590,   1144,
            793,  12100, 235269,    578,    590,   1453, 235303, 235251,   8044,
          24306, 235265,    109,   1969,   2174,  11807,  28294, 235269,    665,
            603,  24048,   1154,    476,  75735, 235265,   1165,    603,  10545,
            675,    573,  25023, 235269,    578,    573,  25023,    603,  14471,
         235265,    109, 159960, 235267,    685,    476,  12425,    575,    573,
           5455,    576,   3354, 235269,  82056,    901,  90892,   1013,   2764,
            476,  26911, 235265,  13227,  14987,    901,   3695,   8829, 235265,
           2625,    970,   7

In [ ]:
gemma_tokenizer.decode(llama_10_input[0])

"<bos><start_of_turn>user\nBelow is a riddle. Come up with 10 more.\nOutput just the riddles. No numbering and don't output anything elseI bubble and laugh and spit water in your face. I am no lady, and I don't wear lace.\n\nAn open ended barrel, it is shaped like a hive. It is filled with the flesh, and the flesh is alive.\n\nStealthy as a shadow in the dead of night, cunning but affectionate if given a bite. Never owned but often loved. At my sport considered cruel, but that's because you never know me at all.\n\nI am a fire's best friend. When fat, my body fills with wind. When pushed to thin, through my nose I blow. Then you can watch the embers glow.\n\nI crawl on the earth. And rise on a pillar.\n\nA box without hinges, lock or key, yet golden treasure lies within. \n\nAs a whole, I am both safe and secure. Behead me, I become a place of meeting. Behead me again, I am the partner of ready. Restore me, I become the domain of beasts.\n\nWho is he that runs without a leg. And his ho

In [ ]:
gemma_10_output = gemma.generate(
    llama_10_input,
    max_new_tokens=500,
    do_sample=True,
    pad_token_id=llama_tokenizer.eos_token_id
)

In [ ]:
output_10 = gemma_tokenizer.decode(gemma_10_output[0], skip_special_tokens=True)
output_10

"user\nBelow is a riddle. Come up with 10 more.\nOutput just the riddles. No numbering and don't output anything elseI bubble and laugh and spit water in your face. I am no lady, and I don't wear lace.\n\nAn open ended barrel, it is shaped like a hive. It is filled with the flesh, and the flesh is alive.\n\nStealthy as a shadow in the dead of night, cunning but affectionate if given a bite. Never owned but often loved. At my sport considered cruel, but that's because you never know me at all.\n\nI am a fire's best friend. When fat, my body fills with wind. When pushed to thin, through my nose I blow. Then you can watch the embers glow.\n\nI crawl on the earth. And rise on a pillar.\n\nA box without hinges, lock or key, yet golden treasure lies within. \n\nAs a whole, I am both safe and secure. Behead me, I become a place of meeting. Behead me again, I am the partner of ready. Restore me, I become the domain of beasts.\n\nWho is he that runs without a leg. And his house on his back?\n\n

In [ ]:
del gemma
torch.cuda.empty_cache()  # this time the model is not offloadin
# restarting the server to release the memory

### Llama observation

- Llama provides the output of 10 riddles

- Llama encoding and decoding is working same as roberta models

In [ ]:
import torch

code_llama_path = "/home/kamal/.cache/huggingface/hub/models--codellama--CodeLlama-7b-hf/snapshots/bc5283229e2fe411552f55c71657e97edf79066c/"
code_tokenizer = AutoTokenizer.from_pretrained(code_llama_path)
codellama = AutoModelForCausalLM.from_pretrained(
    code_llama_path,
    device_map="auto",
    quantization_config=quant_config,
    torch_dtype=torch.bfloat16
)  # takes around 11.5GB of VRAM
# with 4-bit quantization 5GB of VRAM is consumed

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
code_tokenizer.default_chat_template

"{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'] %}{% elif false == true and not '<<SYS>>' in messages[0]['content'] %}{% set loop_messages = messages %}{% set system_message = 'You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\\n\\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\\'t know the answer to a question, please don\\'t share false information.' %}{% else %}{% set loop_messages = messages %}{% set system_message = false %}{% endif %}{% for message in loop_messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must

In [ ]:
simple_request = "Generate 10 riddles that you know"

simple_message = [{"role":"user", "content":simple_request}]

In [ ]:
code_request = "Generate 10 dictionary related problems"

code_message = [{"role":"user", "content":code_request}]

In [ ]:
code_input = code_tokenizer.apply_chat_template(simple_message,return_tensors='pt').to("cuda")

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


In [ ]:
code_input = code_tokenizer.apply_chat_template(code_message,return_tensors='pt').to("cuda")

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


In [ ]:
code_output = codellama.generate(
    code_input,
    max_new_tokens=500,
    do_sample=True,
    pad_token_id=code_tokenizer.eos_token_id
)

In [ ]:
code_output = codellama.generate(
    code_input,
    max_new_tokens=500,
    do_sample=True,
    pad_token_id=code_tokenizer.eos_token_id,
    temperature=0.2
)

In [ ]:
code_output = codellama.generate(
    code_input,
    max_new_tokens=500,
    # do_sample=True,
    pad_token_id=code_tokenizer.eos_token_id,
    # temperature=0.2
)

In [ ]:
code_output = codellama.generate(
    code_input,
    max_new_tokens=500,
    # do_sample=True,
    pad_token_id=code_tokenizer.eos_token_id,
    # temperature=0.2
    repetition_penalty=0.5
)

In [ ]:
code_output = codellama.generate(
    code_input,
    max_new_tokens=500,
    do_sample=True,
    pad_token_id=code_tokenizer.eos_token_id,
    # temperature=0.2
    repetition_penalty=0.5,
    top_p=10,
    top_k=20
)

In [ ]:
output = code_output[0][len(code_input[0]):]
code_tokenizer.decode(output, skip_special_tokens=True)

'ayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaSSLayaayaayaayaayaayaayaSSLayaSSLayaayaayaayaayaayaayaayaayaayaayaayaayaSSLayaayaayaayaayaayaayaayaayaayaSSLayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaSSLayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaSSLayaayaayaayaayaayaSSLayaayaayaayaayaayaayaayaSSLayaSSLayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaSSLayaayaayaayaayaayaSSLayaayaayaayaayaSSLayaayaayaayaayaayaayaayaayaSSLayaayaayaayaayaayaayaayaayaayaayaayaSSLayaayaayaayaayaayaayaSSLayaayaSSLayaSSLayaayaSSLayaayaayaayaayaayaayaayaayaayaayaayaayaayaayaSSLayaayaayaayaayaayaayaayaaya

### Code Llama observation

- Model inference lead to OOM with 500 tokens request, when loaded with bfloat16

- Model inference worked with 500 Tokens, with quant_config done with 4-bit.

- In quantisation 1GB of Vram is consumed for inference


- **The model out was gibberish**

- Requested for the code related to dictionary

- Even after reviewing the generation configs, only gibberish was generated